In [1]:
import pandas as pd
df=pd.read_csv("mergedcanpred.csv")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70627 entries, 0 to 70626
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   roomtype          70627 non-null  object 
 1   Country           47695 non-null  object 
 2   arrival_date      70627 non-null  object 
 3   departure_date    70627 non-null  object 
 4   stayperiod        70627 non-null  int64  
 5   leadtime          70627 non-null  int64  
 6   transaction_date  70627 non-null  object 
 7   adult             70627 non-null  int64  
 8   child             70627 non-null  int64  
 9   Source            50263 non-null  object 
 10  status            70627 non-null  int64  
 11  ADR               70627 non-null  float64
 12  Total             70626 non-null  float64
dtypes: float64(2), int64(5), object(6)
memory usage: 7.0+ MB


In [3]:
df.head(10)

,roomtype,Country,arrival_date,departure_date,stayperiod,leadtime,transaction_date,adult,child,Source,status,ADR,Total
0,SUITE,India,2024-03-23,2024-03-24,1,18,05/03/2024,3,0,NaN,4,7410.72,8300.00
1,DELUXE ROOM,India,2024-03-21,2024-03-24,3,16,05/03/2024,2,0,NaN,4,6250.00,21000.00
2,DELUXE ROOM,India,2024-03-09,2024-03-11,2,4,05/03/2024,3,0,NaN,4,12033.90,28400.00
3,SUITE,NaN,2024-03-05,2024-03-06,1,0,05/03/2024,2,0,MakemytripXml,4,4650.00,5208.00
4,SUITE,NaN,2024-03-19,2024-03-21,2,14,05/03/2024,1,0,NaN,4,7232.14,16200.00
5,DELUXE ROOM,NaN,2024-03-19,2024-03-21,2,14,05/03/2024,1,0,NaN,4,7232.14,16200.00
6,SUITE,India,2024-03-05,2024-03-06,1,0,05/03/2024,2,0,MakemytripXml,1,4650.00,5208.00
7,DELUXE ROOM,India,2024-03-05,2024-03-07,2,0,05/03/2024,1,0,NaN,6,0.00,0.00
8,SHAHPURA SUITE,India,2024-11-08,2024-11-10,2,248,05/03/2024,1,0,NaN,4,7711.86,18200.00
9,ROYAL SUITE,NaN,2024-03-09,2024-03-10,1,4,05/03/2024,2,2,MakemytripXml,4,14349.00,16931.82


In [4]:
df['roomtype'].value_counts()

SUITE             37437
DELUXE ROOM       16619
SHAHPURA SUITE     8453
ROYAL SUITE        7974
BANQUET             144
Name: roomtype, dtype: int64

In [5]:
df['Country'].isnull().sum()

22932

In [6]:
# Merge 'India' and 'INDIA' into a single category
df['Country'] = df['Country'].str.capitalize()  # Ensure consistent capitalization
df['Country'] = df['Country'].replace({'India': 'India', 'india': 'India', 'INDIA': 'India'})

# Replace all other countries with 'Foreign'
df.loc[(~df['Country'].isin(['India'])) & (df['Country'].notnull()), 'Country'] = 'Foreign'

In [7]:
# Convert columns to datetime
df['arrival_date'] = pd.to_datetime(df['arrival_date'])
df['departure_date'] = pd.to_datetime(df['departure_date'])
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

<ipython-input-7-f09ac97f4367>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['transaction_date'] = pd.to_datetime(df['transaction_date'])


In [8]:
# List of date columns to drop
date_columns = ['arrival_date', 'departure_date', 'transaction_date']

# Drop the date columns
df.drop(columns=date_columns, inplace=True)

In [9]:
df.head(5)

,roomtype,Country,stayperiod,leadtime,adult,child,Source,status,ADR,Total
0,SUITE,India,1,18,3,0,NaN,4,7410.72,8300.0
1,DELUXE ROOM,India,3,16,2,0,NaN,4,6250.00,21000.0
2,DELUXE ROOM,India,2,4,3,0,NaN,4,12033.90,28400.0
3,SUITE,NaN,1,0,2,0,MakemytripXml,4,4650.00,5208.0
4,SUITE,NaN,2,14,1,0,NaN,4,7232.14,16200.0


In [10]:
df['Country'] = df['Country'].map({'India': 1, 'Foreign': 0})

In [11]:
df.drop(columns=['child','Total','ADR'], inplace=True)
df.head(5)

,roomtype,Country,stayperiod,leadtime,adult,Source,status
0,SUITE,1.0,1,18,3,NaN,4
1,DELUXE ROOM,1.0,3,16,2,NaN,4
2,DELUXE ROOM,1.0,2,4,3,NaN,4
3,SUITE,NaN,1,0,2,MakemytripXml,4
4,SUITE,NaN,2,14,1,NaN,4


In [12]:
df = df[df['status'] != 4]

# Replace values in the 'status' column
df['status'] = df['status'].replace({2: 1, 6: 0 , 5: 0, 7: 0, 10: 0,11: 0})

# Verify the changes
print(df['status'].value_counts())

1    34616
0    29770
Name: status, dtype: int64


<ipython-input-12-3c7061c8e2fe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status'] = df['status'].replace({2: 1, 6: 0 , 5: 0, 7: 0, 10: 0,11: 0})


In [13]:
df = df.dropna(subset=['Country'])

In [14]:
df['Country'] = df['Country'].astype(int)

In [15]:
ota_list = [
    'MakemytripXml', 'Agoda', 'go-mmt','Booking.com','BOOKING.COM'
]

# Filtered DataFrame containing only OTA records
df = df[df['Source'].isin(ota_list)].replace({'MakemytripXml':'mmt','go-mmt' : 'mmt','BOOKING.COM':'Booking.com'})

In [16]:
from imblearn.over_sampling import RandomOverSampler

# Assuming df is your DataFrame and 'Country' is the target variable
ros = RandomOverSampler(random_state=42)

# Resample the dataset
df, _ = ros.fit_resample(df, df['Country'])
df, _ = ros.fit_resample(df, df['status'])
df, _ = ros.fit_resample(df, df['Source'])
df, _ = ros.fit_resample(df, df['Country'])
df, _ = ros.fit_resample(df, df['status'])

In [17]:
 # Perform one-hot encoding
df_encoded = pd.get_dummies(df['Source'], prefix='Source')

# Concatenate the encoded columns with the original DataFrame
df = pd.concat([df,df_encoded], axis=1)

df.drop('Source', axis=1, inplace=True)

In [18]:
df['Country'].value_counts()

0    35863
1    33895
Name: Country, dtype: int64

In [19]:
df['status'].value_counts()

1    34879
0    34879
Name: status, dtype: int64

In [22]:
df['roomtype'].value_counts()

DELUXE ROOM       32354
SUITE             23384
ROYAL SUITE        8536
SHAHPURA SUITE     5474
BANQUET              10
Name: roomtype, dtype: int64

In [23]:
df.drop(df[df['roomtype'] == 'BANQUET'].index, inplace=True)

# Print the updated DataFrame
df['roomtype'].value_counts()

DELUXE ROOM       32354
SUITE             23384
ROYAL SUITE        8536
SHAHPURA SUITE     5474
Name: roomtype, dtype: int64

In [24]:
roomtype_mapping = {
    'ROYAL SUITE': 3,
    'SHAHPURA SUITE': 4,
    'SUITE': 2,
    'DELUXE ROOM': 1
}

# Map the values in the 'roomtype' column
df['roomtypemap'] = df['roomtype'].map(roomtype_mapping)

# Print the updated DataFrame
df = df.drop('roomtype',axis=1)
df

,Country,stayperiod,leadtime,adult,status,Source_Agoda,Source_Booking.com,Source_mmt,roomtypemap
0,1,1,0,2,1,0,0,1,2
1,0,2,0,1,0,1,0,0,4
2,1,1,1,2,1,1,0,0,1
3,1,1,1,2,1,1,0,0,1
4,1,1,0,1,1,0,0,1,2
...,...,...,...,...,...,...,...,...,...
69753,1,2,25,2,0,0,1,0,1
69754,0,1,77,1,0,1,0,0,1
69755,0,2,143,1,0,0,1,0,3
69756,1,2,8,2,0,0,0,1,2


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X = df.drop('status', axis=1)  # Features
y = df['status']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.linear_model import LogisticRegression

# Initialize and train the logistic regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)


# Initialize logistic regression model
logreg = LogisticRegression()

# Train the model
logreg.fit(X_train, y_train)

# Make predictions
y_pred = logreg.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.74      0.80      0.77      6961
           1       0.78      0.72      0.75      6989

    accuracy                           0.76     13950
   macro avg       0.76      0.76      0.76     13950
weighted avg       0.76      0.76      0.76     13950



In [37]:
import numpy as np

# Define a function to take manual input and predict using the trained model
def predict_status(logistic_model):
    Country = 1
    stayperiod = 2
    lead_time = 60
    adult = 2
    roomtypemap = 3
    Source_Agoda = 0
    Source_Booking_com = 1
    Source_go_mmt = 0

    # Create a numpy array with the input values
    input_data = np.array([[Country, stayperiod, lead_time, adult,roomtypemap, Source_Agoda, Source_Booking_com, Source_go_mmt]])

    # Make prediction using the trained model
    prediction = logistic_model.predict(input_data)

    return prediction
predict_status(logistic_model)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0])

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error

X = df.drop(columns=['status'])  # Drop the 'status' column from features
y = df['status']  # Select the 'status' column as the target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)



# Assuming y_true and y_pred are arrays of true and predicted values, respectively
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE is the square root of MSE


In [38]:
print(mae,mse,rmse)

0.3715674389504028 0.179038058359421 0.4231288909533607


In [33]:
import numpy as np

# Define a function to take manual input and predict using the trained model
def predict_status(model):
    Country = 1
    stayperiod = 2
    lead_time = 60
    adult = 2
    roomtypemap = 3
    Source_Agoda = 0
    Source_Booking_com = 1
    Source_go_mmt = 0

    # Create a numpy array with the input values
    input_data = np.array([[Country, stayperiod, lead_time, adult,roomtypemap, Source_Agoda, Source_Booking_com, Source_go_mmt]])

    # Make prediction using the trained model
    prediction = model.predict(input_data)

    return prediction
predict_status(model)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([0.17885962])